In [101]:
from flask import Flask
from flask import  render_template,request
from flask import jsonify
import os
import sys
import scipy
import numpy as np
import pandas as pd
from sklearn import metrics, preprocessing
from modules import ml_create_target_df, get_top_probable_tags, mean_local_score, f1_local_score_accuracy, text_processing, text_processing_advanced, local_score
from sklearn.externals import joblib

current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
# I. On récupère les données utilisateur
# title_input = request.form['Title']
# body_input = request.form['Body']
# title_input = str(title_input)
# body_input = str(body_input)

title_input = "zezezezezeze zezezez"
body_input = "qdfqsdfsdf sdfsdfsdf sdfsdflokooksdfki "

body_title_input = title_input + " " + body_input

# II. Nettoyage	du texte.
body_title_input_cleaned = text_processing(body_title_input)

# III. Chargement des variables utiles liées à notre jeu d'entrainement
vectorizer_train = joblib.load(parent_path + "/pipeline/vectorizer.pkl")
count_tags_df = joblib.load(parent_path + "/pipeline/count_tags_df.pkl")
most_popular_tag = joblib.load(parent_path + "/pipeline/most_popular_tag.pkl")
unique_tags_serie = joblib.load(parent_path + "/pipeline/unique_tags_serie.pkl")
unique_tags = joblib.load(parent_path + "/pipeline/unique_tags.pkl")
train_selected_tags = joblib.load(parent_path + "/pipeline/selected_tags.pkl")
list_top_tags = joblib.load(parent_path + "/pipeline/list_top_tags.pkl")

# IV. Chargement des classifieurs (logistical regression)
training_classifiers = []
for tag in train_selected_tags:
    training_classifiers.append(joblib.load(parent_path + "/classifiers/logistical_regression/logreg_" + tag +".pkl"))

body_title_input_cleaned = body_title_input_cleaned.split("0")

# V. Application du vectorizer_train de notre training
X_input = vectorizer_train.transform(body_title_input_cleaned)

# VI. Prédiction de la probabilité
print("Prédiction de la probabilité pour chaque question d'appartenir à un Tag donné en cours de traitement...")
X_input_predictions = []
count = 1
zero_array = np.zeros((X_input.shape[0]))
for clf in training_classifiers:
    prediction = clf.predict_proba(X_input)

    if prediction.shape[1] == 2:
        X_input_predictions.append(prediction[:,1])
    elif prediction.shape[1] == 1:
        X_input_predictions.append(zero_array)
    count += 1

# threshold
trusted_threshold = 0.11

# # Map threshold
X_input_predictions_df = pd.DataFrame(np.array(X_input_predictions).T, columns = train_selected_tags)
X_input_predictions_thresh_df = X_input_predictions_df.copy()
X_input_predictions_thresh_df[X_input_predictions_thresh_df<trusted_threshold]= 0


# VII. Seuil

yourtags = get_top_probable_tags(X_input_predictions_thresh_df.iloc[0], train_selected_tags)

return render_template("resultats.html" , TAGS = yourtags)

Prédiction de la probabilité pour chaque question d'appartenir à un Tag donné en cours de traitement...


In [102]:
yourtags

'c#'

In [91]:
X_input_predictions_thresh_df[X_input_predictions_thresh_df<0.1]= 0

In [92]:
X_input_predictions_thresh_df

Tag,c#,.net,java,asp.net,javascript,c++,php,python,sql,sql-server,...,.net-3.5,bash,entity-framework,forms,stored-procedures,reporting-services,firefox,visual-studio-2005,visual-c++,sockets
0,0.170001,0.108653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
# [0 if el<trusted_threshold else el for el in X_input_predictions]
for n, i in enumerate(X_input_predictions):
    if i <0.11:
        X_input_predictions[n] = 0

In [73]:
pd.DataFrame(np.array(X_input_predictions).T, columns = train_selected_tags)

[array([0.17000076]),
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10]

In [74]:
X_input_predictions_thresh_df = pd.DataFrame(np.array(X_input_predictions).T, columns = train_selected_tags)

ValueError: Shape of passed values is (1, 99), indices imply (99, 99)